**Load Module**

In [1]:
# Utils
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import os
import time
from datetime import datetime
from sklearn.model_selection import StratifiedShuffleSplit
from utility.utils import EarlyStopping, WarmUpLR, most_recent_folder, most_recent_weights, best_acc_weights, last_epoch

# Torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
import torch.optim as optim
from torch.utils.data import DataLoader, SubsetRandomSampler, Subset

import torchvision
from torchvision import transforms, datasets

# Analysis
from sklearn.metrics import confusion_matrix, classification_report
import wandb


**Seed & Device Setting**

In [2]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # GPU 여러 개 사용할 경우 사용
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False # 완벽한 재현성을 위해선 False로 하는 게 맞지만 성능의 저하를 일으킬 수 있음
    np.random.seed(seed)
    random.seed(seed)

seed = 42
set_seed(seed)

# 디바이스 설정
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

**Wandb Setting & Set Hyperparameters (수정 구간)**

In [3]:
config = {
    "batch_size" : 128,
    "num_epochs" : 250,
    "learning_rate" : 0.1,
    "momentum" : 0.9,
    "weight_decay" : 5e-4,
    "nesterov" : True,
    "gamma" : 0.2,
    "warm" : 1,
    "patience" : 150,
    "plateau_patience" : 15,
    "milestones" : [100, 150, 200],
    "pin_memory" : True,

    "resume" : False,
    "model_name" : "resnet_18_base_50000+nesterov",
    "stratified_data" : True,
    "scheduler" : "MultiStepLR",
    "train_50000" : True
}
if config["resume"] == True:
    config["warm"] = 0
model_name = config["model_name"]
wandb.init(project="CIFAR-100_Classification", name=config["model_name"], config=config)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: hyunwook7120 (hyunwook7120-hanyang-university). Use `wandb login --relogin` to force relogin


In [4]:
fine_to_superclass = {
    4: 0, 30: 0, 55: 0, 72: 0, 95: 0,       # aquatic mammals
    1: 1, 32: 1, 67: 1, 73: 1, 91: 1,       # fish
    54: 2, 62: 2, 70: 2, 82: 2, 92: 2,      # flowers
    9: 3, 10: 3, 16: 3, 28: 3, 61: 3,       # food containers
    0: 4, 51: 4, 53: 4, 57: 4, 83: 4,       # fruit and vegetables
    22: 5, 39: 5, 40: 5, 86: 5, 87: 5,      # household electrical devices
    5: 6, 20: 6, 25: 6, 84: 6, 94: 6,       # household furniture
    6: 7, 7: 7, 14: 7, 18: 7, 24: 7,        # insects
    3: 8, 42: 8, 43: 8, 88: 8, 97: 8,       # large carnivores
    12: 9, 17: 9, 37: 9, 68: 9, 76: 9,      # large man-made outdoor things
    23: 10, 33: 10, 49: 10, 60: 10, 71: 10, # large natural outdoor scenes
    15: 11, 19: 11, 21: 11, 31: 11, 38: 11, # large omnivores and herbivores
    34: 12, 63: 12, 64: 12, 66: 12, 75: 12, # medium-sized mammals
    26: 13, 45: 13, 77: 13, 79: 13, 99: 13, # non-insect invertebrates
    2: 14, 11: 14, 35: 14, 46: 14, 98: 14,  # people
    27: 15, 29: 15, 44: 15, 78: 15, 93: 15, # reptiles
    36: 16, 50: 16, 65: 16, 74: 16, 80: 16, # small mammals
    47: 17, 52: 17, 56: 17, 59: 17, 96: 17, # trees
    8: 18, 13: 18, 48: 18, 58: 18, 90: 18,  # vehicles 1
    41: 19, 69: 19, 81: 19, 85: 19, 89: 19  # vehicles 2
}

# Data Preprocessing

**Data Augementation (수정 구간)**

In [5]:
CIFAR100_TRAIN_MEAN = (0.5070751592371323, 0.48654887331495095, 0.4409178433670343)
CIFAR100_TRAIN_STD = (0.2673342858792401, 0.2564384629170883, 0.27615047132568404)

train_val_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    # transforms.RandomRotation(degrees=15),
    # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR100_TRAIN_MEAN, CIFAR100_TRAIN_STD)
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR100_TRAIN_MEAN, CIFAR100_TRAIN_STD)
])

**Splitting the training data**

In [6]:
train_val_data = datasets.CIFAR100(root='./data', train=True, download=True, transform=train_val_transform)
val_data = datasets.CIFAR100(root='./data', train=True, download=True, transform=test_transform)
test_data = datasets.CIFAR100(root='./data', train=False, download=True, transform=test_transform)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [7]:
# Stratified 방식으로 train 데이터를 train/val로 나누기
if config["train_50000"]:
   pass
elif config["stratified_data"]:
  labels = train_val_data.targets
  stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

  for train_index, val_index in stratified_split.split(train_val_data.data, labels):
      train_data = Subset(train_val_data, train_index)
      val_data = Subset(val_data, val_index)
else:
  # RandomSampler 방식으로 train 데이터를 train/val로 나누기
  num_train = len(train_val_data)
  indices = list(range(num_train))
  split = int(np.floor(0.2 * num_train))  # validation 데이터를 20%로 설정

  np.random.shuffle(indices)
  train_idx, val_idx = indices[split:], indices[:split]

  train_sampler = SubsetRandomSampler(train_idx)
  val_sampler = SubsetRandomSampler(val_idx)

**Define Dataloader**

In [8]:
if config["train_50000"]:
  train_loader = DataLoader(train_val_data, batch_size=config["batch_size"], shuffle=True, num_workers=4, pin_memory=config["pin_memory"])
  test_loader = DataLoader(test_data, batch_size=config["batch_size"], shuffle=False, num_workers=4, pin_memory=config["pin_memory"])
elif config["stratified_data"]:
  train_loader = DataLoader(train_data, batch_size=config["batch_size"], shuffle=True, num_workers=4, pin_memory=config["pin_memory"])
  val_loader = DataLoader(val_data, batch_size=config["batch_size"], shuffle=False, num_workers=4, pin_memory=config["pin_memory"])
  test_loader = DataLoader(test_data, batch_size=config["batch_size"], shuffle=False, num_workers=4, pin_memory=config["pin_memory"])
else:
  train_loader = DataLoader(train_val_data, batch_size=config["batch_size"], sampler=train_sampler, num_workers=4, pin_memory=config["pin_memory"])
  val_loader = DataLoader(train_val_data, batch_size=config["batch_size"], shuffle=False, sampler=val_sampler, num_workers=4, pin_memory=config["pin_memory"])
  test_loader = DataLoader(test_data, batch_size=config["batch_size"], shuffle=False, num_workers=4, pin_memory=config["pin_memory"])

**Model Initialize (수정 부분)**

In [9]:
from models import resnet

print("use:", device)

# 모델 초기화
model = resnet.resnet18()
net = model
# 모델을 GPU로 이동
net.to(device)

# print(summary(net, (3,224,224)))

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=config["learning_rate"], momentum=config["momentum"], weight_decay=config["weight_decay"], nesterov=config["nesterov"])

if config["scheduler"] == "MultiStepLR":
    train_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, config["milestones"], gamma=config["gamma"]) #learning rate decay
elif config["scheduler"] == "ReduceLROnPlateau":
    train_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min", factor=config["gamma"], patience=config["plateau_patience"])
else:
    print("No scheduler!!")
iter_per_epoch = len(train_loader)
warmup_scheduler = WarmUpLR(optimizer, iter_per_epoch * config["warm"])
early_stopping = EarlyStopping(patience=config["patience"], verbose=True)

print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

use: cuda:0
Optimizer's state_dict:
state 	 {}
param_groups 	 [{'lr': 0.0, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0.0005, 'nesterov': True, 'maximize': False, 'foreach': None, 'differentiable': False, 'fused': None, 'initial_lr': 0.1, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61]}]


**Model Train**

In [10]:
# 모델 학습 함수
def train_model(net, trainloader, criterion, optimizer, epoch):
    net.train()
    start = time.time()
    top1_correct = 0
    top5_correct = 0
    total = 0

    superclass_correct = 0
    superclass_total = 0

    running_loss = 0.0
    for batch_idx, (inputs, labels) in enumerate(trainloader):
        labels, inputs = labels.to(device), inputs.to(device)

        optimizer.zero_grad()

        # 모델 예측
        outputs = net(inputs)
        loss = criterion(outputs, labels)

        _, preds = torch.max(outputs, 1)
        top1_correct += torch.sum(preds == labels).item()

        _, top5_preds = outputs.topk(5, 1, True, True)
        top5_correct += torch.sum(top5_preds.eq(labels.view(-1, 1).expand_as(top5_preds))).item()

        total += labels.size(0)

        # Superclass accuracy
        super_preds = torch.tensor([fine_to_superclass[p.item()] for p in preds], dtype=torch.long)
        super_labels = torch.tensor([fine_to_superclass[t.item()] for t in labels], dtype=torch.long)
        superclass_correct += torch.sum(super_preds == super_labels).item()
        superclass_total += super_labels.size(0)

        # 역전파 및 최적화
        loss.backward()
        optimizer.step()

        n_iter = (epoch - 1) * len(train_loader) + batch_idx + 1
        running_loss += loss.item() * inputs.size(0)

        # 30번째 배치마다 상태 출력
        if (batch_idx + 1) % 30 == 0:
            print(f"Batch [{batch_idx+1}/{len(trainloader)}], Loss: {loss.item():.4f}")

        # LR에 warmup 되는 게 안 보이지만 실제론 이루어지고 있음. batch로 돌 때 이루어짐.
        if epoch <= config["warm"]:
            warmup_scheduler.step()


    # Epoch당 평균 손실 계산 및 출력
    epoch_loss = running_loss / total

    num_epochs = config["num_epochs"]
    finish = time.time()

    # 세부 클래스 및 슈퍼 클래스 정확도 계산
    top1_acc = top1_correct / total
    top5_acc = top5_correct / total
    superclass_acc = superclass_correct / superclass_total
    accuracy = [top1_acc, top5_acc, superclass_acc]

    print("Train set: Epoch: {}, Average loss:{:.4f}, LR: {:.6f} Top-1 Accuracy: {:.4f}, Top-5 Accuracy: {:.4f}, SuperClass Accuracy: {:.4f}, Time consumed:{:.2f}s".format(
        epoch,
        epoch_loss,
        optimizer.param_groups[0]['lr'],
        top1_acc,
        top5_acc,
        superclass_acc,
        finish - start
    ))

    return epoch_loss, accuracy

def eval_training(net, testloader, criterion, epoch):
    net.eval()
    top1_correct = 0
    top5_correct = 0
    total = 0

    superclass_correct = 0
    superclass_total = 0

    start = time.time()

    test_loss = 0.0

    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)

            _, preds = torch.max(outputs, 1)
            top1_correct += torch.sum(preds == labels).item()

            _, top5_preds = outputs.topk(5, 1, True, True)
            top5_correct += torch.sum(top5_preds.eq(labels.view(-1, 1).expand_as(top5_preds))).item()

            total += labels.size(0)

            test_loss += criterion(outputs, labels)

            # Superclass accuracy
            super_preds = torch.tensor([fine_to_superclass[p.item()] for p in preds], dtype=torch.long)
            super_labels = torch.tensor([fine_to_superclass[t.item()] for t in labels], dtype=torch.long)
            superclass_correct += torch.sum(super_preds == super_labels).item()
            superclass_total += super_labels.size(0)

    finish = time.time()
    average_loss = test_loss / total

    # 세부 클래스 및 슈퍼 클래스 정확도 계산
    top1_acc = top1_correct / total
    top5_acc = top5_correct / total
    superclass_acc = superclass_correct / superclass_total
    accuracy = [top1_acc, top5_acc, superclass_acc]

    print("Test set: Epoch: {}, Average loss:{:.4f}, Top-1 Accuracy: {:.4f}, Top-5 Accuracy: {:.4f}, SuperClass Accuracy: {:.4f}, Time consumed:{:.2f}s".format(
        epoch,
        average_loss,
        top1_acc,
        top5_acc,
        superclass_acc,
        finish - start
    ))
    print()

    return average_loss, accuracy

In [11]:
DATE_FORMAT = '%A_%d_%B_%Y_%Hh_%Mm_%Ss'
TIME_NOW = datetime.now().strftime(DATE_FORMAT)

if config["resume"]:
    recent_folder = most_recent_folder(os.path.join("runs", config["model_name"], "savepoints"), DATE_FORMAT)
    if not recent_folder:
        raise Exception("no recent folder were found")

    checkpoint_path = os.path.join("runs", config["model_name"], "savepoints", recent_folder)
else:
    checkpoint_path = os.path.join("runs", config["model_name"], "savepoints", TIME_NOW)

if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)
checkpoint_path = os.path.join(checkpoint_path, '{net}-{epoch}-{type}.pth')

best_acc = [0.0, 0.0, 0.0]
sum_best_acc = 0.0
epoch = 0
weights_path = False

if config["resume"]:
    best_weights = best_acc_weights(os.path.join("runs", config["model_name"], "savepoints", recent_folder))
    if best_weights:
        weights_path = os.path.join("runs", config["model_name"], "savepoints", recent_folder, best_weights)
        print('found best acc weights file:{}'.format(weights_path))
        print('load best training file to test acc...')
        net.load_state_dict(torch.load(weights_path))
        _, best_acc = eval_training(net, test_loader, criterion, epoch)
        print("best acc is Top-1:{:0.2f} Top-5:{:0.2f} Super:{:0.2f} Total:{:0.2f}".format(best_acc[0], best_acc[1], best_acc[2], sum(best_acc)))

    recent_weights_file = most_recent_weights(os.path.join("runs", config["model_name"], "savepoints", recent_folder))
    if not recent_weights_file:
        raise Exception("no recent weights file were found")
    weights_path = os.path.join("runs", config["model_name"], "savepoints", recent_folder, recent_weights_file)
    print("loading weights file {} to resume training......".format(weights_path))
    net.load_state_dict(torch.load(weights_path))

    resume_epoch = last_epoch(os.path.join("runs", config["model_name"], "savepoints", recent_folder))
    print("resume_epoch is", resume_epoch)
    optimizer = optim.SGD(net.parameters(), lr=config["learning_rate"], momentum=config["momentum"], weight_decay=config["weight_decay"], nesterov=config["nesterov"])

    if config["scheduler"] == "MultiStepLR":
        train_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, config["milestones"], gamma=config["gamma"]) #learning rate decay
    elif config["scheduler"] == "ReduceLROnPlateau":
        train_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min", factor=config["gamma"], patience=config["plateau_patience"])
    else:
        print("No scheduler!!")
    optimizer.step()

**Per-Epoch Training**

In [12]:
start_time = time.time()
for epoch in tqdm(range(1, config["num_epochs"] + 1)):
    if epoch > config["warm"]:
        if config["scheduler"] == "MultiStepLR":
            train_scheduler.step()
        elif config["scheduler"] == "ReduceLROnPlateau":
            train_scheduler.step(test_loss)

    if config["resume"]:
        if epoch <= resume_epoch:
            continue

    epoch_loss, train_accuracy = train_model(net, train_loader, criterion, optimizer, epoch=epoch)
    if config["train_50000"] == False:
        test_loss, test_accuracy = eval_training(net, val_loader, criterion, epoch)
    else:
        test_loss, test_accuracy = eval_training(net, test_loader, criterion, epoch)

    if sum_best_acc < sum(test_accuracy):
        if weights_path:  # 모델의 용량 이슈로 새로운 best 모델이 갱신되면 이전 best 모델 삭제
            os.remove(weights_path)
        weights_path = checkpoint_path.format(net=config["model_name"], epoch=epoch, type="best")
        print("saving weights file to {}".format(weights_path))
        torch.save(net.state_dict(), weights_path)
        best_acc = test_accuracy
        sum_best_acc = sum(best_acc)

    early_stopping(test_loss, net)
    if early_stopping.early_stop:
        print("Early stopping")
        break

    wandb.log({
        "epoch": epoch,
        "learning_rate": optimizer.param_groups[0]['lr'],
        "train_loss": epoch_loss,
        "train top-1 accuracy": train_accuracy[0],
        "train top-5 accuracy": train_accuracy[1],
        "train super-class accuracy": train_accuracy[2],
        "val_loss": test_loss,
        "valid top-1 accuracy": test_accuracy[0],
        "valid top-5 accuracy": test_accuracy[1],
        "valid super-class accuracy": test_accuracy[2]
    })

end_time = time.time()
wandb.log({"train_time": end_time - start_time})

# Result
print(f"Result of best {model_name} = Epoch : {epoch}   Loss : {test_loss:.4f}   Top-1 Accuracy : {best_acc[0]*100:.4f}%  Top-5 Accuracy : {best_acc[1]*100:.4f}%   Super Accuracy : {best_acc[2]*100:.4f}%   Total_Accuracy : {sum(best_acc)*100:.4f}    Time : {end_time - start_time:.4f}")

  0%|          | 0/250 [00:00<?, ?it/s]

Batch [30/391], Loss: 4.5003
Batch [60/391], Loss: 3.8105
Batch [90/391], Loss: 3.9335
Batch [120/391], Loss: 3.9804
Batch [150/391], Loss: 3.6056
Batch [180/391], Loss: 3.6764
Batch [210/391], Loss: 3.5094
Batch [240/391], Loss: 3.5642
Batch [270/391], Loss: 3.4500
Batch [300/391], Loss: 3.5599
Batch [330/391], Loss: 3.3586
Batch [360/391], Loss: 3.0189
Batch [390/391], Loss: 3.1685
Train set: Epoch: 1, Average loss:3.7572, LR: 0.100000 Top-1 Accuracy: 0.1242, Top-5 Accuracy: 0.3470, SuperClass Accuracy: 0.2466, Time consumed:44.78s
Test set: Epoch: 1, Average loss:0.0283, Top-1 Accuracy: 0.1624, Top-5 Accuracy: 0.4312, SuperClass Accuracy: 0.2948, Time consumed:13.06s

saving weights file to runs\resnet_18_base_50000+nesterov\savepoints\Sunday_13_October_2024_15h_09m_35s\resnet_18_base_50000+nesterov-1-best.pth


  0%|          | 1/250 [00:57<4:00:17, 57.90s/it]

Batch [30/391], Loss: 2.9745
Batch [60/391], Loss: 3.2021
Batch [90/391], Loss: 3.0298
Batch [120/391], Loss: 2.9485
Batch [150/391], Loss: 2.7804
Batch [180/391], Loss: 2.7980
Batch [210/391], Loss: 2.3938
Batch [240/391], Loss: 2.9347
Batch [270/391], Loss: 2.4134
Batch [300/391], Loss: 2.5878
Batch [330/391], Loss: 2.7083
Batch [360/391], Loss: 2.5674
Batch [390/391], Loss: 2.5683
Train set: Epoch: 2, Average loss:2.8411, LR: 0.100000 Top-1 Accuracy: 0.2760, Top-5 Accuracy: 0.5934, SuperClass Accuracy: 0.4269, Time consumed:45.00s
Test set: Epoch: 2, Average loss:0.0229, Top-1 Accuracy: 0.2791, Top-5 Accuracy: 0.5986, SuperClass Accuracy: 0.4336, Time consumed:13.52s

saving weights file to runs\resnet_18_base_50000+nesterov\savepoints\Sunday_13_October_2024_15h_09m_35s\resnet_18_base_50000+nesterov-2-best.pth


  1%|          | 2/250 [01:56<4:00:58, 58.30s/it]

...

Batch [30/391], Loss: 0.0094
Batch [60/391], Loss: 0.0107
Batch [90/391], Loss: 0.0104
Batch [120/391], Loss: 0.0085
Batch [150/391], Loss: 0.0093
Batch [180/391], Loss: 0.0080
Batch [210/391], Loss: 0.0113
Batch [240/391], Loss: 0.0131
Batch [270/391], Loss: 0.0096
Batch [300/391], Loss: 0.0115
Batch [330/391], Loss: 0.0088
Batch [360/391], Loss: 0.0081
Batch [390/391], Loss: 0.0117
Train set: Epoch: 249, Average loss:0.0100, LR: 0.000800 Top-1 Accuracy: 0.9997, Top-5 Accuracy: 1.0000, SuperClass Accuracy: 0.9999, Time consumed:44.62s
Test set: Epoch: 249, Average loss:0.0068, Top-1 Accuracy: 0.7868, Top-5 Accuracy: 0.9431, SuperClass Accuracy: 0.8696, Time consumed:15.26s

saving weights file to runs\resnet_18_base_50000+nesterov\savepoints\Sunday_13_October_2024_15h_09m_35s\resnet_18_base_50000+nesterov-249-best.pth
EarlyStopping counter: 21 out of 150


100%|█████████▉| 249/250 [4:17:41<00:58, 58.09s/it]

Batch [30/391], Loss: 0.0099
Batch [60/391], Loss: 0.0086
Batch [90/391], Loss: 0.0120
Batch [120/391], Loss: 0.0130
Batch [150/391], Loss: 0.0140
Batch [180/391], Loss: 0.0088
Batch [210/391], Loss: 0.0115
Batch [240/391], Loss: 0.0083
Batch [270/391], Loss: 0.0060
Batch [300/391], Loss: 0.0123
Batch [330/391], Loss: 0.0086
Batch [360/391], Loss: 0.0128
Batch [390/391], Loss: 0.0076
Train set: Epoch: 250, Average loss:0.0100, LR: 0.000800 Top-1 Accuracy: 0.9997, Top-5 Accuracy: 1.0000, SuperClass Accuracy: 0.9999, Time consumed:44.71s
Test set: Epoch: 250, Average loss:0.0068, Top-1 Accuracy: 0.7883, Top-5 Accuracy: 0.9421, SuperClass Accuracy: 0.8705, Time consumed:13.08s

saving weights file to runs\resnet_18_base_50000+nesterov\savepoints\Sunday_13_October_2024_15h_09m_35s\resnet_18_base_50000+nesterov-250-best.pth
EarlyStopping counter: 22 out of 150


100%|██████████| 250/250 [4:18:39<00:00, 62.08s/it]


Result of best resnet_18_base_50000+nesterov = Epoch : 250   Loss : 0.0068   Top-1 Accuracy : 78.8300%  Top-5 Accuracy : 94.2100%   Super Accuracy : 87.0500%   Total_Accuracy : 260.0900    Time : 15519.4174


**Best Model Test**

In [13]:
def all_accuracy(net, test_loader, device):
    net2.eval()
    top1_correct = 0
    top5_correct = 0
    total = 0

    superclass_correct = 0
    superclass_total = 0

    with torch.no_grad():
        for _, (images, labels) in enumerate(test_loader):

            images, labels = images.to(device), labels.to(device)
            outputs = net2(images)

            _, preds = torch.max(outputs, 1)
            top1_correct += torch.sum(preds == labels).item()

            _, top5_preds = outputs.topk(5, 1, True, True)
            top5_correct += torch.sum(top5_preds.eq(labels.view(-1, 1).expand_as(top5_preds))).item()

            total += labels.size(0)

            # Superclass accuracy
            super_preds = torch.tensor([fine_to_superclass[p.item()] for p in preds], dtype=torch.long)
            super_labels = torch.tensor([fine_to_superclass[t.item()] for t in labels], dtype=torch.long)
            superclass_correct += torch.sum(super_preds == super_labels).item()
            superclass_total += super_labels.size(0)

    # 세부 클래스 및 슈퍼 클래스 정확도 계산
    top1_acc = top1_correct / total
    top5_acc = top5_correct / total
    superclass_acc = superclass_correct / superclass_total
    accuracy = [top1_acc, top5_acc, superclass_acc]

    return accuracy

In [14]:
net2 = model
net2.to(device)
net2.eval()

# 이 코드로 best 모델이 load 되지 않을 경우 아래 코드에서 경로 직접 지정
recent_folder = most_recent_folder(os.path.join("runs", config["model_name"], "savepoints"), DATE_FORMAT)
if not recent_folder:
    raise Exception("no recent folder were found")

best_weights = best_acc_weights(os.path.join("runs", config["model_name"], "savepoints", recent_folder))
if best_weights:
    weights_path = os.path.join("runs", config["model_name"], "savepoints", recent_folder, best_weights)
    print('found best acc weights file:{}'.format(weights_path))
    print('load best training file to test acc...')
    net2.load_state_dict(torch.load(weights_path))

acc = all_accuracy(net2, test_loader, device)
print("Top-1 accuracy: {:.4f}".format(acc[0]))
print("Top-5 accuracy: {:.4f}".format(acc[1]))
print("Super-Class accuracy: {:.4f}".format(acc[2]))

wandb.log({
    "Test Top-1 accuracy": acc[0],
    "Test Top-5 accuracy": acc[1],
    "Test Super-Class accuracy": acc[2],
    "Total Score": sum(acc)
    })

found best acc weights file:runs\resnet_18_base_50000+nesterov\savepoints\Sunday_13_October_2024_15h_09m_35s\resnet_18_base_50000+nesterov-250-best.pth
load best training file to test acc...


C:\Users\jhw03\AppData\Local\Temp\ipykernel_20140\1591997829.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net2.load_state_dict(torch.load(weights_path))


Top-1 accuracy: 0.7883
Top-5 accuracy: 0.9421
Super-Class accuracy: 0.8705


In [15]:
# # 위 코드에서 에러가 발생해 경로 지정이 제대로 되지 않을 경우 사용

# net2 = model
# net2.to(device)
# net2.load_state_dict(torch.load("runs/wide_resnet/savepoints/Tuesday_01_October_2024_13h_15m_52s/wide_resnet-229-best.pth"))

# acc = all_accuracy(net2, test_loader, device)
# print("Top-1 accuracy: {:.4f}".format(acc[0]))
# print("Top-5 accuracy: {:.4f}".format(acc[1]))
# print("Super-Class accuracy: {:.4f}".format(acc[2]))
# wandb.log({
#     "Test Top-1 accuracy": acc[0],
#     "Test Top-5 accuracy": acc[1],
#     "Test Super-Class accuracy": acc[2],
#     "Total Score": sum(acc)
#     })
# wandb.finish()

**Data Analysis**

In [16]:
import wandb.sklearn

classes = train_val_data.classes
coarse_classes = [
    'aquatic mammals', 'fish', 'flowers', 'food containers', 'fruit and vegetables', 'household electrical devices',
    'household furniture', 'insects', 'large carnivores', 'large man-made outdoor things',
    'large natural outdoor scenes', 'large omnivores and herbivores', 'medium-sized mammals',
    'non-insect invertebrates', 'people', 'reptiles', 'small mammals', 'trees', 'vehicles 1', 'vehicles 2'
]

y_pred = []
y_true = []
y_probs = []

for x, y in torch.utils.data.DataLoader(dataset=test_data, batch_size=config["batch_size"]):
    x = x.to(device)
    y = y.to(device)
    z = net2(x)
    _, yhat = torch.max(z, 1)
    probs = z.softmax(dim=1).cpu().detach().numpy()

    pred = yhat.data.cpu().numpy()
    
    super_pred = [coarse_classes[fine_to_superclass[p]] for p in pred]
    y_pred.extend(super_pred)

    labels = y.data.cpu().numpy()
    super_labels = [coarse_classes[fine_to_superclass[l]] for l in labels]
    y_true.extend(super_labels)

    y_probs.extend(probs)
    
cf_matrix = confusion_matrix(y_true, y_pred, normalize='true')

df_cm = pd.DataFrame(cf_matrix, index=coarse_classes,
                     columns=coarse_classes)

plt.figure(figsize=(10, 7))
sns.heatmap(df_cm, annot=True, fmt='.2f', cmap='Blues')
plt.title(f'Confusion Matrix of {config["model_name"]} (CIFAR-100 Superclass)')
plt.xlabel('Predicted')
plt.ylabel('Real')

conf_matrix_img = wandb.Image(plt.gcf())
plt.close()

class_report = classification_report(y_true, y_pred, labels=coarse_classes, zero_division=0)

plt.figure(figsize=(10, 7))
plt.text(0.01, 0.05, str(class_report), {'fontsize': 12}, fontproperties='monospace')  # monospaced font
plt.axis('off')  # 축 제거
plt.tight_layout()

class_report_img = wandb.Image(plt.gcf())
plt.close()

wandb.log({
    "Confusion Matrix": conf_matrix_img,
    "Classification Report": class_report_img
    })

In [17]:
wandb.finish()

Test Super-Class accuracy,▁
Test Top-1 accuracy,▁
Test Top-5 accuracy,▁
Total Score,▁
epoch,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▇▇▇▇▇▇▇▇██
learning_rate,█████████████████▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train super-class accuracy,▁▂▃▃▃▄▄▄▄▄▄▄▄▄▇▇▇▇▇▇▇▇▇▇▇███████████████
train top-1 accuracy,▁▃▄▄▄▅▅▅▅▅▅▅▅▅▅▅▅▅▅▇▇█▇▇▇███████████████
train top-5 accuracy,▁▂▃▄▅▅▅▆▅▆▆▆▆▆▆▆▆▆▆▆▆███████████████████
train_loss,█▆▆▆▆▆▆▆▆▆▆▆▆▅▅▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_time,▁
